In [ ]:
pip install reka-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.3
    Uninstalling httpx-sse-0.4.3:
      Successfully uninstalled httpx-sse-0.4.3


In [ ]:
import requests
import json
import time

API_KEY = "439f1a07e89c2dc44d13bac65a7132a7d8b044cee49d7bfced7005d7afdbf82a"
VISION_BASE = "https://vision-agent.api.reka.ai"

headers = {"X-Api-Key": API_KEY}

# ---- OPTION A: Reuse your existing video_id (already uploaded!) ----
video_id = "6a85643f-d9de-479b-a69a-3d0ba273b9f0"  # from your last run

# ✅ STEP 1: Check if it's done indexing now
status_resp = requests.get(f"{VISION_BASE}/v1/videos/{video_id}", headers=headers)
print("Current status:", status_resp.json().get("indexing_status"))

# ✅ STEP 2: Wait until indexed (bumped to 20 attempts, 15s apart = 5 mins total)
for attempt in range(20):
    status_resp = requests.get(f"{VISION_BASE}/v1/videos/{video_id}", headers=headers)
    status = status_resp.json().get("indexing_status")
    print(f"Attempt {attempt+1}: status = {status}")

    if status == "indexed":
        print(" Ready for QA!")
        break
    elif status == "failed":
        print("❌ Indexing failed. Re-upload needed.")
        break

    time.sleep(15)

# ✅ STEP 3: QA — CORRECT endpoint is /qa/chat (not /videos/qa)
if status == "indexed":
    qa_resp = requests.post(
        f"{VISION_BASE}/v1/qa/chat",  # ← THIS WAS THE BUG
        headers={**headers, "Content-Type": "application/json"},
        json={
            "video_id": video_id,
            "messages": [
                {
                    "role": "user",
                    "content": """Analyze this video as a viral content director. Output ONLY valid JSON:
{
  "hook": "first 3 seconds description",
  "energy": "low/medium/high",
  "pacing": "slow/medium/fast",
  "visual_style": "cinematic/raw/polished/lo-fi",
  "location_cues": ["list of visible locations"],
  "brand_placement_opportunities": ["where logos could naturally appear"],
  "dominant_emotion": "excitement/trust/curiosity/urgency",
  "suggested_kling_prompt": "video generation prompt based on this style"
}"""
                }
            ]
        }
    )
    print("QA Result:", json.dumps(qa_resp.json(), indent=2))

Current status: indexed
Attempt 1: status = indexed
 Ready for QA!
QA Result: {
  "chat_response": "{\n  \"hook\": \"A young man with red hair in a formal suit singing passionately into a vintage-style microphone against a window-pane background.\",\n  \"energy\": \"high\",\n  \"pacing\": \"fast\",\n  \"visual_style\": \"cinematic\",\n  \"location_cues\": [\n    \"indoor studio with window-pane backdrop\",\n    \"urban concrete wall walkway\",\n    \"dark alleyway\",\n    \"white fence and arched doorway\",\n    \"bright outdoor sunny area\"\n  ],\n  \"brand_placement_opportunities\": [\n    \"vintage microphone casing\",\n    \"lapel of the singer's suit\",\n    \"graphics on the woman's yellow top\",\n    \"branding on the boy's white jacket\",\n    \"logo on the concrete wall in the outdoor shot\"\n  ],\n  \"dominant_emotion\": \"excitement\",\n  \"suggested_kling_prompt\": \"Cinematic music video style, rapid cuts between a red-haired male singer in a striped shirt and suit singing

In [ ]:
import requests
import json
import time

API_KEY = "439f1a07e89c2dc44d13bac65a7132a7d8b044cee49d7bfced7005d7afdbf82a"
VISION_BASE = "https://vision-agent.api.reka.ai"
CHAT_BASE = "https://api.reka.ai"
TAVILY_KEY = "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"

headers = {"X-Api-Key": API_KEY}

# ============================================================
# STEP 1: Tavily — Find trending SF tech content
# ============================================================
tavily_resp = requests.post(
    "https://api.tavily.com/search",
    headers={"Content-Type": "application/json"},
    json={
        "api_key": TAVILY_KEY,
        "query": "Claude AI dominating San Francisco tech scene 2025",
        "search_depth": "advanced",
        "include_images": True,
        "max_results": 5
    }
)
tavily_data = tavily_resp.json()
print("Tavily status:", tavily_resp.status_code)
print("Tavily found:", len(tavily_data.get("results", [])), "results")

# Extract top 3 article summaries
context_snippets = [r["content"] for r in tavily_data.get("results", [])[:3]]
context = "\n\n".join(context_snippets)
print("\nContext preview:", context[:300], "...\n")

# ============================================================
# STEP 2: Reka Chat — Generate Director's Brief from Tavily context
# ============================================================
brief_resp = requests.post(
    f"{CHAT_BASE}/v1/chat",
    headers={**headers, "Content-Type": "application/json"},
    json={
        "model": "reka-flash",
        "messages": [
            {
                "role": "user",
                "content": f"""You are a viral video director. Based on this trending news about Claude AI in SF:

{context}

Generate a Director's Brief for a 5-second viral video that places OpenAI branding
in a San Francisco scene where Claude is currently winning.
Output ONLY valid JSON, no markdown, no code fences:
{{
  "concept": "one sentence video concept",
  "hook": "what viewer sees in first 3 seconds",
  "sf_scene": "specific SF location (e.g. Ferry Building, SOMA billboard, Caltrain)",
  "brand_placement": "exactly where OpenAI logo appears naturally in scene",
  "kling_prompt": "detailed 2-3 sentence Kling video generation prompt",
  "predicted_emotion": "what viewer feels",
  "virality_score": "1-10"
}}"""
            }
        ]
    }
)

print("Reka status:", brief_resp.status_code)
brief_json = brief_resp.json()

# Handle Reka response format
if "choices" in brief_json:
    raw_brief = brief_json["choices"][0]["message"]["content"]
elif "responses" in brief_json:
    raw_brief = brief_json["responses"][0]["message"]["content"]
else:
    print("Unexpected Reka response:", json.dumps(brief_json, indent=2))
    raw_brief = ""

print("\nRaw brief from Reka:\n", raw_brief)

# Parse to clean JSON
clean = raw_brief.replace("```json", "").replace("```", "").strip()
director_brief = json.loads(clean)

print("\nDIRECTOR'S BRIEF:")
print(json.dumps(director_brief, indent=2))
print("\nKLING PROMPT READY:")
print(director_brief["kling_prompt"])

Tavily status: 200
Tavily found: 5 results

Context preview: Databricks shows what enterprise AI looks like at scale. Data platform revenue run rate reached $5.4billion by the end of 2025 and increased by more than 65 percent annually. Over one million dollars are spent on more than 800 customers each year. The company has a good free cash flow and constructs ...

Reka status: 200

Raw brief from Reka:
  ```json
{
  "concept": "OpenAI's Claude triumphs in the heart of San Francisco's tech revolution.",
  "hook": "A bustling crowd in SoMa cheers as a giant digital screen displays Claude's success metrics.",
  "sf_scene": "South of Market (SOMA) district with towering tech company billboards.",
  "brand_placement": "An OpenAI logo subtly integrated into the digital screen's corner, glowing with a soft blue light.",
  "kling_prompt": "Capture a dynamic scene of a jubilant crowd in San Francisco's SOMA, focusing on a massive digital billboard celebrating Claude's victories, with the OpenAI

In [ ]:
# STEP 1: Tavily finds trending content
tavily_resp = requests.post(
    "https://api.tavily.com/search",
    headers={"Content-Type": "application/json"},
    json={
        "api_key": "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",
        "query": "Claude AI dominating San Francisco tech scene 2024",
        "search_depth": "advanced",
        "include_images": True,
        "max_results": 5
    }
)
trending_data = tavily_resp.json()
print("Trending results:", json.dumps(trending_data, indent=2))

# STEP 2: Feed Tavily results into Reka to generate Director's Brief
context = "\n".join([r["content"] for r in trending_data.get("results", [])[:3]])

brief_resp = requests.post(
    "https://api.reka.ai/v1/chat",
    headers=headers,
    json={
        "model": "reka-core",  # Use core for better reasoning
        "messages": [
            {
                "role": "user",
                "content": f"""Based on this trending content about Claude AI in SF:

{context}

Generate a Director's Brief for a viral TikTok/Instagram video that subtly places OpenAI branding
where Claude is currently winning. Output ONLY valid JSON:
{{
  "concept": "one sentence video concept",
  "hook": "first 3 seconds - what viewer sees",
  "scene": "specific SF location to use",
  "brand_placement": "exactly where/how OpenAI logo appears naturally",
  "kling_prompt": "detailed video generation prompt ready for Kling API",
  "predicted_emotion": "what viewer feels",
  "estimated_virality": "1-10 score with reason"
}}"""
            }
        ]
    }
)
brief = brief_resp.json()
print("Director's Brief:", json.dumps(brief, indent=2))

Trending results: {
  "query": "Claude AI dominating San Francisco tech scene 2024",
  "follow_up_questions": null,
  "answer": null,
  "images": [
    "https://storage.googleapis.com/nucamp-production.appspot.com/aiseo-blogs/coding-bootcamp-san-francisco-ca/coding-bootcamp-san-francisco-ca-how-to-stay-updated-with-san-franciscos-evolving-tech-industry-trends/thumbnail01.webp",
    "https://www.aleaitsolutions.com/wp-content/uploads/2024/09/AI-Powered-Startups-and-Innovation-in-San-Franciscos-Tech-Scene.png",
    "https://media.licdn.com/dms/image/v2/D5622AQH98hPcBKCrbw/feedshare-shrink_2048_1536/feedshare-shrink_2048_1536/0/1727771475875?e=2147483647&v=beta&t=LQGCOGfhc4eA9ypQPWfmu0W4iBLyZdV3eFTulNDrLM4",
    "https://blog.ahuraai.com/wp-content/uploads/2024/11/Ahura-AI-San-Francisco-Tech-Week-Recap-768x433.png",
    "https://cdn.prod.website-files.com/62c65fa04fedf7630780fe3c/64123136c614f5f110b594c4_6412311e968b261033a7b77b_wDAdl51.webp"
  ],
  "results": [
    {
      "url": "https: